In [260]:
import pandas as pd
import numpy as np

In [261]:
info = pd.read_csv('../data/cleaned/info.csv')
info = info.set_index('dbn')
directory = pd.read_csv('../data/2020_DOE_High_School_Directory.csv')

metrics = pd.read_csv('../data/metrics_2019.csv')
metrics = metrics.set_index('dbn')

school_names = pd.read_csv('../data/cleaned/school_names.csv')
school_names = school_names.set_index('dbn')

In [262]:
folder = '../data/cleaned/'

applicants = pd.read_csv(folder + 'applicants.csv')
applicants = applicants.set_index('dbn')

avg_applicants = pd.read_csv(folder + 'avg_applicants.csv')
avg_applicants = avg_applicants.set_index('dbn')

min_applicants = pd.read_csv(folder + 'min_applicants.csv')
min_applicants = min_applicants.set_index('dbn')

max_applicants = pd.read_csv(folder + 'max_applicants.csv')
max_applicants = max_applicants.set_index('dbn')

offers = pd.read_csv(folder + 'offers.csv')
offers = offers.set_index('dbn')

avg_offers = pd.read_csv(folder + 'avg_offers.csv')
avg_offers = avg_offers.set_index('dbn')

min_offers = pd.read_csv(folder + 'min_offers.csv')
min_offers = min_offers.set_index('dbn')

max_offers = pd.read_csv(folder + 'max_offers.csv')
max_offers = max_offers.set_index('dbn')

In [263]:
cols = ['Total', 'Asian','Black','Hispanic','White']

### Offer Rates

#### Individual Schools

In [264]:
avg_offers.div(avg_applicants)[cols]

,Total,Asian,Black,Hispanic,White
dbn,,,,,
01M450,0.063135,0.038328,0.057927,0.079946,0.087542
01M539,0.065936,0.072747,0.027565,0.037964,0.095463
01M696,0.070259,0.082908,0.036411,0.043887,0.101432
02M047,0.211587,0.147059,0.156250,0.228571,0.208333
02M260,0.057597,0.041872,0.028436,0.041485,0.087282
...,...,...,...,...,...
28Q690,0.282787,0.342105,0.297030,0.213333,0.147059
28Q896,0.223062,0.294118,0.216374,0.160000,0.108696
30Q286,0.176367,0.211921,0.113208,0.151515,0.317460


#### All Schools

In [265]:
avg_offers.sum().div(avg_applicants.sum())[cols]

Total       0.084898
Asian       0.089531
Black       0.077703
Hispanic    0.078455
White       0.104258
dtype: float64

#### Top Ranking Schools

In [266]:
top = metrics[metrics.eq('Far Above Average').sum(axis=1).gt(4)].index
print(len(top))
top = [dbn for dbn in top if dbn in info.index]
print(len(top))

31
27


In [267]:
avg_offers.loc[top].sum().div(avg_applicants.loc[top].sum())[cols]

Total       0.068399
Asian       0.085978
Black       0.044055
Hispanic    0.048914
White       0.091636
dtype: float64

In [283]:
df_info = pd.DataFrame([['All Screened Schools', len(offers), applicants['Total'].sum(), offers['Total'].sum()],
              ['"Top-Ranked" Screened Schools', len(offers.loc[top]), applicants.loc[top]['Total'].sum(), offers.loc[top]['Total'].sum()]], columns=['index','No. of Schools', 'No. of Applicants', 'No. of Offers'])
df_info = df_info.set_index('index')

df = pd.DataFrame()
df['All Screened Schools'] = avg_offers.sum().div(avg_applicants.sum()).mul(100).round(1)[cols]
df['"Top-Ranked" Screened Schools'] = avg_offers.loc[top].sum().div(avg_applicants.loc[top].sum()).mul(100).round(1)[cols]
df = df.T
df.columns = [col.replace('Hispanic', 'Latino').replace('Total','Overall') + " Offer Rate (%)" for col in df.columns]

display(df)

df_info.join(df, how='outer').to_csv('../output/all-vs-top-ranked-rates.csv')
df_info.join(df, how='outer')

,Overall Offer Rate (%),Asian Offer Rate (%),Black Offer Rate (%),Latino Offer Rate (%),White Offer Rate (%)
All Screened Schools,8.5,9.0,7.8,7.8,10.4
"""Top-Ranked"" Screened Schools",6.8,8.6,4.4,4.9,9.2


,No. of Schools,No. of Applicants,No. of Offers,Overall Offer Rate (%),Asian Offer Rate (%),Black Offer Rate (%),Latino Offer Rate (%),White Offer Rate (%)
All Screened Schools,75,181713,15427,8.5,9.0,7.8,7.8,10.4
"""Top-Ranked"" Screened Schools",27,107326,7341,6.8,8.6,4.4,4.9,9.2


In [269]:
for col in cols:
    count = avg_offers.loc[top].sum()[col]
    nobs  = avg_applicants.loc[top].sum()[col]
    prop_var = min_offers.loc[top].sum()[col] / max_applicants.loc[top].sum()[col]
    value = 0.05
    stats, pval = proportions_ztest(count, nobs, value, prop_var = prop_var)
    print(col, '{0:0.3f}'.format(pval))

    prop_var = max_offers.loc[top].sum()[col] / min_applicants.loc[top].sum()[col]
    stats, pval = proportions_ztest(count, nobs, value, prop_var = prop_var)
    print(col, '{0:0.3f}'.format(pval))
    
    print(round(avg_offers.loc[top].sum()[col] / avg_applicants.loc[top].sum()[col] * 100, 1),
          round(min_offers.loc[top].sum()[col] / max_applicants.loc[top].sum()[col] * 100, 1),
          round(max_offers.loc[top].sum()[col] / min_applicants.loc[top].sum()[col] * 100, 1))

Total 0.000
Total 0.000
6.8 6.8 6.8
Asian 0.000
Asian 0.000
8.6 8.6 8.6
Black 0.000
Black 0.000
4.4 4.3 4.6
Hispanic 0.389
Hispanic 0.392
4.9 4.9 4.9
White 0.000
White 0.000
9.2 9.1 9.2


### Demographic Breakdowns of Applicants and Offers

#### Individual Schools

In [275]:
redacted_offers = offers.eq('s') | offers.eq('s^')
redacted_applicants = applicants.eq('s') | applicants.eq('s^')

df = pd.DataFrame()
df['School Name'] = school_names['school_name']
df['# Applicants'] = avg_applicants['Total']
df[['% Asian Applicants', '% Black Applicants', '% Latino Applicants', '% White Applicants']] = avg_applicants[['Asian', 'Black', 'Hispanic', 'White']].div(avg_applicants['Total'],axis='index').mul(100).round(1)
df[['% Asian Applicants - Estimate Off By (+/-)', '% Black Applicants - Estimate Off By (+/-)', '% Latino Applicants - Estimate Off By (+/-)', '% White Applicants - Estimate Off By (+/-)']] = max_applicants.sub(avg_applicants).div(avg_applicants['Total'], axis='index').mul(100).round(1)[['Asian', 'Black','Hispanic','White']]
df[['# Offers']] = avg_offers['Total']
df[['% Asian Offers', '% Black Offers', '% Latino Offers', '% White Offers']] = avg_offers[['Asian', 'Black', 'Hispanic', 'White']].div(avg_offers['Total'],axis='index').mul(100).round(1)
df[['% Asian Offers - Estimate Off By (+/-)', '% Black Offers - Estimate Off By (+/-)', '% Latino Offers - Estimate Off By (+/-)', '% White Offers - Estimate Off By (+/-)']] = max_offers.sub(avg_offers).div(avg_offers['Total'], axis='index').mul(100).round(1)[['Asian', 'Black','Hispanic','White']]

column_order = ['School Name']
column_order.extend(sorted([col for col in df if 'Applicants' in col]))
column_order.extend(sorted([col for col in df if 'Offers' in col]))

df[column_order].to_csv('../output/pct-applicants-offers-stats.csv')
df[column_order]

,School Name,# Applicants,% Asian Applicants,% Asian Applicants - Estimate Off By (+/-),% Black Applicants,% Black Applicants - Estimate Off By (+/-),% Latino Applicants,% Latino Applicants - Estimate Off By (+/-),% White Applicants,% White Applicants - Estimate Off By (+/-),# Offers,% Asian Offers,% Asian Offers - Estimate Off By (+/-),% Black Offers,% Black Offers - Estimate Off By (+/-),% Latino Offers,% Latino Offers - Estimate Off By (+/-),% White Offers,% White Offers - Estimate Off By (+/-)
dbn,,,,,,,,,,,,,,,,,,,
01M450,East Side Community School,1869.0,15.4,0.0,17.5,0.0,39.5,0.0,15.9,0.0,118.0,9.3,0.0,16.1,0.0,50.0,0.0,22.0,0.0
01M539,"New Explorations into Science, Technology and ...",6203.0,29.7,0.0,10.5,0.0,18.7,0.0,27.4,0.0,409.0,32.8,0.0,4.4,0.0,10.8,0.0,39.6,0.0
01M696,Bard High School Early College,3971.0,19.7,0.0,19.4,0.0,24.1,0.0,21.1,0.0,279.0,23.3,0.0,10.0,0.0,15.1,0.0,30.5,0.0
02M047,47 The American Sign Language and English Seco...,397.0,4.3,0.0,24.2,0.0,61.7,0.0,3.0,0.0,84.0,3.0,3.0,17.9,0.0,66.7,0.0,3.0,3.0
02M260,The Clinton School,2830.0,14.3,0.0,7.5,0.0,16.2,0.0,42.5,0.0,163.0,10.4,0.0,3.7,0.0,11.7,0.0,64.4,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28Q690,High School for Law Enforcement and Public Safety,488.0,15.6,0.0,41.4,0.0,30.7,0.0,3.5,0.0,138.0,18.8,0.0,43.5,0.0,23.2,0.0,1.8,1.8
28Q896,"Young Women's Leadership School, Queens",529.0,32.1,0.0,32.3,0.0,18.9,0.0,4.3,0.0,118.0,42.4,0.0,31.4,0.0,13.6,0.0,2.1,2.1
30Q286,"Young Women's Leadership School, Astoria",567.0,26.6,0.0,9.3,0.0,40.7,0.0,11.1,0.0,100.0,32.0,0.0,6.0,0.0,35.0,0.0,20.0,0.0


#### Combined: All Schools and Top Ranking Schools

In [271]:
df = pd.DataFrame()

top = metrics[metrics.eq('Far Above Average').sum(axis=1).gt(4)].index
top = [dbn for dbn in top if dbn in info.index]

groups = [('Top-Ranked Screened High Schools', top), 
          ('All Screened High Schools', info.index)]
rows = []

for (title, dbns) in groups:
    row = []
    a = avg_applicants.loc[dbns].sum()[['Total', 'Asian', 'Black', 'Hispanic', 'White']]
    o = avg_offers.loc[dbns].sum()[['Total', 'Asian', 'Black', 'Hispanic', 'White']]
    arr = max_applicants.loc[dbns].sum()[['Total', 'Asian', 'Black', 'Hispanic', 'White']]
    #print(arr - a)
    orr = max_offers.loc[dbns].sum()[['Total', 'Asian', 'Black', 'Hispanic', 'White']]
    print((arr - a).div(a['Total']).mul(100).round(1))
    print()
    print(count)
    
    row.append(title)
    row.append(len(dbns))
    row.append(a['Total'])
    row.extend(list(a[['Asian', 'Black', 'Hispanic', 'White']].div(a['Total']).mul(100).round(1)))
    row.extend(list((arr - a).div(a['Total']).mul(100).round(1)[['Asian', 'Black', 'Hispanic', 'White']]))
    row.append(o['Total'])
    row.extend(list(o[['Asian', 'Black', 'Hispanic', 'White']].div(o['Total']).mul(100).round(1)))
    row.extend(list((orr - o).div(o['Total']).mul(100).round(1)[['Asian', 'Black', 'Hispanic', 'White']]))

    rows.append(row)
    
columns = [
    'Category',
    '# Schools',
    '# Applicants',
    '% Asian Applicants', 
    '% Black Applicants', 
    '% Latino Applicants', 
    '% White Applicants',
    '% Asian Applicants - Estimate Off By (+/-)', 
    '% Black Applicants - Estimate Off By (+/-)', 
    '% Latino Applicants - Estimate Off By (+/-)', 
    '% White Applicants - Estimate Off By (+/-)',
    '# Offers',
    '% Asian Offers',
    '% Black Offers',
    '% Latino Offers',
    '% White Offers',
    '% Asian Offers - Estimate Off By (+/-)', 
    '% Black Offers - Estimate Off By (+/-)', 
    '% Latino Offers - Estimate Off By (+/-)', 
    '% White Offers - Estimate Off By (+/-)'
]

col_order = ['Category', '# Schools']
col_order.extend(sorted([col for col in columns if 'Applicants' in col]))
col_order.extend(sorted([col for col in columns if 'Offers' in col]))

df = pd.DataFrame(rows, columns=columns)[col_order]
df.to_csv('../output/all-vs-top-ranked-shares.csv', index=False)
df

Total       0.0
Asian       0.0
Black       0.0
Hispanic    0.0
White       0.0
dtype: float64

1868.0
Total       0.0
Asian       0.0
Black       0.0
Hispanic    0.0
White       0.0
dtype: float64

1868.0


,Category,# Schools,# Applicants,% Asian Applicants,% Asian Applicants - Estimate Off By (+/-),% Black Applicants,% Black Applicants - Estimate Off By (+/-),% Latino Applicants,% Latino Applicants - Estimate Off By (+/-),% White Applicants,% White Applicants - Estimate Off By (+/-),# Offers,% Asian Offers,% Asian Offers - Estimate Off By (+/-),% Black Offers,% Black Offers - Estimate Off By (+/-),% Latino Offers,% Latino Offers - Estimate Off By (+/-),% White Offers,% White Offers - Estimate Off By (+/-)
0,Top-Ranked Screened High Schools,27,107326.0,24.8,0.0,16.5,0.0,27.2,0.0,19.0,0.0,7341.0,31.2,0.0,10.7,0.4,19.4,0.1,25.4,0.1
1,All Screened High Schools,75,181713.0,18.6,0.0,21.7,0.0,32.4,0.0,16.1,0.0,15427.0,19.6,0.3,19.9,0.2,29.9,0.1,19.8,0.4


In [272]:
pd.DataFrame(rows, columns=columns)[col_order].set_index('Category')

,# Schools,# Applicants,% Asian Applicants,% Asian Applicants - Estimate Off By (+/-),% Black Applicants,% Black Applicants - Estimate Off By (+/-),% Latino Applicants,% Latino Applicants - Estimate Off By (+/-),% White Applicants,% White Applicants - Estimate Off By (+/-),# Offers,% Asian Offers,% Asian Offers - Estimate Off By (+/-),% Black Offers,% Black Offers - Estimate Off By (+/-),% Latino Offers,% Latino Offers - Estimate Off By (+/-),% White Offers,% White Offers - Estimate Off By (+/-)
Category,,,,,,,,,,,,,,,,,,,
Top-Ranked Screened High Schools,27,107326.0,24.8,0.0,16.5,0.0,27.2,0.0,19.0,0.0,7341.0,31.2,0.0,10.7,0.4,19.4,0.1,25.4,0.1
All Screened High Schools,75,181713.0,18.6,0.0,21.7,0.0,32.4,0.0,16.1,0.0,15427.0,19.6,0.3,19.9,0.2,29.9,0.1,19.8,0.4


In [274]:
# "Top-Ranked" Screened Schools Demographics
d = pd.DataFrame()
a = avg_applicants.loc[top].sum()
a = a.rename({'Hispanic':'Latino'})
o = avg_offers.loc[top].sum()
o = o.rename({'Hispanic':'Latino'})
d['Applicants (%)'] = a.div(a['Total']).mul(100).round(1)[['Asian', 'Black', 'Latino', 'White']]
d['Offers (%)'] = o.div(o['Total']).mul(100).round(1)[['Asian', 'Black', 'Latino', 'White']]
d['Difference (%)'] = d['Offers (%)'].sub(d['Applicants (%)']).round(1)
d.to_csv('../output/top-ranked-demo-differences.csv')
display(d)

,Applicants (%),Offers (%),Difference (%)
Asian,24.8,31.2,6.4
Black,16.5,10.7,-5.8
Latino,27.2,19.4,-7.8
White,19.0,25.4,6.4
